In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib 
import numpy as np

In [ ]:
im = Image.open("/content/sample_image.jpg").convert("L")



FileNotFoundError: ignored

In [ ]:
def toArrayAndSquare(im):

 img1 = [[0 for x in range(im.size[0])] for y in range(im.size[1])]
 sqr = [[0 for x in range(im.size[0])] for y in range(im.size[1])]
 for i in range (0,im.size[0]):
  for j in range (0,im.size[1]):
   img1[j][i] = im.getpixel((i,j))
   #img2[j][i] = im.getpixel((i,j))
   sqr[j][i] = img1[j][i] ** 2
  

 return img1, sqr

  
 
  
  
  

In [ ]:
def calculateCooccurrence(im):
 img1, sqr = toArrayAndSquare(im)
 co_occurrence_1 = [[0 for x in range(256)] for y in range(256)] 
 co_occurrence_2 = [[0 for x in range(256)] for y in range(256)] 
 for y in range (0,len(img1)-1):
  for  x in range (0,len(img1[0])):
   i= img1[y][x]
   j= img1[y+1][x]
   co_occurrence_1[i][j] +=1
 for y in range (0,len(img1)):
  for  x in range (0,len(img1[0])-1):
   i= img1[y][x]
   j= img1[y][x+1]
   co_occurrence_2[i][j] +=1
 return co_occurrence_1, co_occurrence_2


In [ ]:
def nullifyPixels(cooccurrence_matrix):
 co_occurrence_null = [[0 for x in range(len(cooccurrence_matrix))] for y in range(len(cooccurrence_matrix[0]))]
 for y in range (0,len(co_occurrence_null)):
   for x in range(0,len(co_occurrence_null[0])):
    co_occurrence_null[y][x] = cooccurrence_matrix[y][x]
 c=0
 for x in range (0,len(co_occurrence_null)):
  for y in range(0,15):
   co_occurrence_null[x][x] = 0
   if(x+1+y<len(co_occurrence_null)):
    co_occurrence_null[x][x+1+y] =0
    c = c+1 
   if(x-1-y>=0):
    co_occurrence_null[x][x-y-1] =0
    c = c+1
   if(c>30):
    continue
 return co_occurrence_null


In [ ]:
def imgWithCooccurrence(cooccurrence_1null, cooccurrence_2null,im):
 constructed_img, sqr= toArrayAndSquare(im)
 for y in range (0,len(constructed_img)-1):
  for  x in range (0,len(constructed_img[0])-1):
   i= constructed_img[y][x]
   j= constructed_img[y+1][x]
   k= constructed_img[y][x+1]
   if(cooccurrence_1null[i][j] == 0 or cooccurrence_2null[i][k] == 0):
    constructed_img[y][x] = 0
   else:
    constructed_img[y][x] = 255
 return constructed_img


  



In [ ]:
co1, co2 = calculateCooccurrence(im)

null1 = nullifyPixels(co1)
null2 = nullifyPixels(co2)

result = imgWithCooccurrence(null1,null2,im)
img1,z = toArrayAndSquare(im)
plt.subplots(nrows=3, ncols=2, figsize=(25, 25))

plt.subplot(3,2,3)
plt.imshow(co1,  cmap = 'gray')

plt.subplot(3,2,5)
plt.imshow(co2,  cmap = 'gray')

plt.subplot(3,2,4)
plt.imshow(null1,  cmap = 'gray')

plt.subplot(3,2,6)
plt.imshow(null2,  cmap = 'gray')

plt.subplot(3,2,2)
plt.imshow(result,  cmap = 'gray')

plt.subplot(3,2,1)
plt.imshow(img1,  cmap = 'gray')






In [ ]:
def integralArray(img_array):
 for x in range (0,len(img_array)):
  sum = 0
  for y in range (0,len(img_array[0])):
   sum = sum + img_array[x][y]
   img_array[x][y] = img_array[x][y] + sum

 for x in range (0,len(img_array[0])):
  sum = 0
  for y in range (0,len(img_array)):
   
   sum = sum + img_array[y][x]
   img_array[y][x] =  sum

 return img_array


In [ ]:

def localSum(img_array, indextop, indexbot):
  sum2=0
  sum2 = img_array[indexbot[0]][indexbot[1]] 
  if(indextop[1] !=0 and indextop[0] ==0):
   sum2 = sum2 - img_array[indexbot[0]][indextop[1]-1]
   
  if(indextop[0] !=0 and indextop[1] ==0):
   sum2 = sum2 - img_array[indextop[0]-1][indexbot[1]]
   
  if(indextop[0] != 0 and indextop[1] != 0):
   sum2 = sum2 - img_array[indexbot[0]][indextop[1]-1]
   sum2 = sum2 - img_array[indextop[0]-1][indexbot[1]]
   sum2 = sum2 + img_array[indextop[0]-1][indextop[1]-1]
  

  return(sum2)




In [ ]:
def applyThreshold (im, th):
  res = [[0 for x in range(len(im[0]))] for y in range(len(im))]
  for i in range (0,len(im)):
    for j in range (0,len(im[0])):
      if(im[i][j]<th):
        res[i][j] = 0
      else:
        res[i][j] = 255
  return res

In [ ]:
from re import X
from numpy.ma.core import var

def imgWithIntegral(im,window_size,with_threshold):
 img1,sqr= toArrayAndSquare(im)
 result,xx = toArrayAndSquare(im)
 center_pixel= window_size[0]//2
 cols= len(img1[0])
 rows= len(img1)
 d=window_size[0]-1
 f=window_size[1]-1
 img1= integralArray(img1)
 sqr= integralArray(sqr)
 #print(img1[50][5])
 #print(sqr[3][40])

 for y in range (center_pixel, rows-center_pixel):
   
  for  x in range (center_pixel,cols-center_pixel):
   
   m= y-center_pixel 
   n= x-center_pixel
   d= y+center_pixel
   f= x+center_pixel

   window_top_left= [m,n]
   window_bot_right= [d,f] 

   

   
   local_sum= localSum(img1,window_top_left,window_bot_right )
   #print(local_sum)
   local_sum_sqr=localSum(sqr,window_top_left,window_bot_right)
   #print(local_sum_sqr)
   one_over_n= 1/(window_size[0]*window_size[1])
   
  
   variance= (one_over_n* local_sum_sqr) - (one_over_n * local_sum)**2
   #print(variance)

   if(variance <0):
    variance=0
   
   if(with_threshold==1):
    threshold2= 750
    
   

    if(variance > threshold2):
     result[y][x] = 255
    
    
    else:
     result[y][x] = 0
   else:
    result[y][x] = variance

   #print(variance)
  
 return result

In [ ]:
plt.subplots(nrows=3, ncols=2, figsize=(25, 25))

image,image_squared = toArrayAndSquare(im)
plt.subplot(3,2,1)
plt.imshow(image,  cmap = 'gray')

plt.subplot(3,2,3)
plt.imshow(image_squared,  cmap = 'gray')

output_integral_1 = integralArray(image)
output_integral_1squared = integralArray(image_squared)

final_result1 = imgWithIntegral(im,[3,3],1)

final_result2 = imgWithIntegral(im,[3,3],0)




plt.subplot(3,2,2)
plt.imshow(output_integral_1,  cmap = 'gray')



plt.subplot(3,2,4)
plt.imshow(output_integral_1squared,  cmap = 'gray')

plt.subplot(3,2,5)
plt.imshow(final_result2,  cmap = 'gray')

plt.subplot(3,2,6)
plt.imshow(final_result1,  cmap = 'gray')
